In [97]:
import requests
import numpy as np
import pandas as pd
import urllib

In [2]:
#### standard post requests######
# defining the api-endpoint  
API_ENDPOINT = "http://pastebin.com/api/api_post.php"
  
# your API key here 
API_KEY = "XXXXXXXXXXXXXXXXX"
  
# your source code here 
source_code = ''' 
print("Hello, world!") 
a = 1 
b = 2 
print(a + b) 
'''
  
# data to be sent to api 
data = {'api_dev_key':API_KEY, 
        'api_option':'paste', 
        'api_paste_code':source_code, 
        'api_paste_format':'python'} 
  
# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data) 
  
# extracting response text  
pastebin_url = r.text 
print("The pastebin URL is:%s"%pastebin_url) 

The pastebin URL is:Bad API request, invalid api_dev_key


In [3]:
####### standard get requst#####
# api-endpoint 
URL = "http://maps.googleapis.com/maps/api/geocode/json"
  
# location given here 
location = "delhi technological university"
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = {'address':location} 
  
# sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 
  
# extracting data in json format 
data = r.json() 
  
  
print(data)

{'error_message': 'You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account', 'results': [], 'status': 'REQUEST_DENIED'}


In [100]:
import json
import re


request_header_string = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''



def parse_string_to_dict(request_header_string):
    request_header_dict={}
    request_header_list = [header.split(": ") if len(header.split(": ")) == 2 else [header.split(": ")[0],header.split(': ')[1]+': '+header.split(': ')[2]] for header in request_header_string.split('\n')]
    for (dict_key,dict_value) in request_header_list:
        request_header_dict[dict_key] = dict_value
    return request_header_dict


In [102]:
URL = "http://192.168.0.10/read_file.php?filename=defaultParameters.txt&time=0"
  
  
# defining a params dict for the parameters to be sent to the API 
PARAMS = parse_string_to_dict(request_header_string)
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 
  
response.text

'"1350\\n2450\\n1\\n16044\\n10\\n1100\\n51\\n1726\\n" \n\n'

In [ ]:
"1350\n2450\n1\n16044\n10\n1100\n51\n1726\n" 

# Calibration script

In [151]:
### read file script 1
header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''


PARAMS = parse_string_to_dict(header)
URL = "http://192.168.0.10/read_file.php?filename=calibration_data%2Fpixel_coeffs.txt&time=0"
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 
  
response.text


'"-1650.2637939453125000000000000000000000000000000000000000000000000000\\n1.1157624721527099609375000000000000000000000000000000000000000000\\n0.0001071036967914551496505737304687500000000000000000000000000000\\n" \n\n'

In [152]:
#### write file script 1
n_random= np.random.randint(10,300)

write_file_body={"a":''' exec 1> log.txt
rm patterns/*.csv
rm patterns/*.bmp
rm patterns/*.sdf
rm tmp/*
rm *.bmp
echo "Generating Patterns..." > tmp/script_status{n}.txt
dlp_nirscan -A51 -Z1726 -N110
if [ $? -eq 255 ];
 then
 echo "failed" > tmp/script_status{n}.txt
 exit 0
 fi
cd patterns/
echo "Preparing Scan Solution..." > ../tmp/script_status10.txt
dlp_nirscan -Pscan.sdf
if [ $? -eq 255 ];
 then
 echo "failed" > ../tmp/script_status{n}.txt
 exit 0
 fi
echo "Loading Patterns..." > ../tmp/script_status{n}.txt
count=0
for entry in *
do
case $entry in *.bmp)
convert $entry -virtual-pixel Black  -filter point -interpolate nearestneighbor -distort polynomial "2 $(cat /usr/share/matrix-gui-2.0/calibration_data/control_points.txt)" scan_img.bmp
mv scan_img.bmp $entry
count=$((count+1));;
esac
done
dlp_nirscan -l$count

cd .. 
dlp_nirscan -S110 -E1400 -fcustom_scan
#Using a more unique string to detect if the script is completed
echo "_?!!SCRIPT_COMPLETED!!?_" > tmp/script_status{n}.txt'''.format(n=n_random),
'filename': 'scan.sh'}



header_string = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Content-Length: 1185
Content-Type: application/x-www-form-urlencoded
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Origin: http://192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''

header = parse_string_to_dict(header_string)

data = write_file_body
API_ENDPOINT = "http://192.168.0.10/write_file.php"
# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data, headers=header) 
  
# extracting response text  
print(r.text)

In [153]:
### read file script 1
header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''


PARAMS = parse_string_to_dict(header)
URL = "http://192.168.0.10/run_scan.php"
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 
  
response.text


''

In [154]:
#### write file script 2
body = {'a': '''1350
2450
10
1610
10
110
51
1726''',
'filename': 'defaultParameters.txt'}



header_string = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Content-Length: 82
Content-Type: application/x-www-form-urlencoded
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Origin: http://192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''

header = parse_string_to_dict(header_string)
data = body

API_ENDPOINT = "http://192.168.0.10/write_file.php"
# sending post request and saving response as response object 
r = requests.post(url = API_ENDPOINT, data = data, headers=header) 
  
# extracting response text  
print(r.text)

In [155]:
### check ccalibration status###
### read file script 1
import time
header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''


PARAMS = parse_string_to_dict(header)
URL = "http://192.168.0.10/tmp/script_status{n}.txt".format(n=n_random)
  
# sending get request and saving the response as response object 
response = requests.get(url = URL, params = PARAMS) 

while response.status_code == 404:
    print(response.text)
    time.sleep(2)
    response = requests.get(url = URL, params = PARAMS)
response.text


'_?!!SCRIPT_COMPLETED!!?_\n'

In [5]:
# n_random
header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''

In [102]:
class Data:
    def encoded(self,params):
        return urllib.parse.urlencode(params, doseq=True)
        

In [142]:
class Parser:
    def parse_string_to_dict(request_header_string):
        request_header_dict={}
        request_header_list = [header.split(": ") if len(header.split(": ")) == 2 else [header.split(": ")[0],header.split(': ')[1]+': '+header.split(': ')[2]] for header in request_header_string.split('\n')]
        for (dict_key,dict_value) in request_header_list:
            request_header_dict[dict_key] = dict_value
        return request_header_dict
class Header:
    standard_header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36
X-Requested-With: XMLHttpRequest'''
    
    post_php_header = '''Accept: */*
Accept-Encoding: gzip, deflate
Accept-Language: en,th;q=0.9
Connection: keep-alive
Content-Length: 1191
Content-Type: application/x-www-form-urlencoded
Cookie: iconGridCol=4; iconGridRow=3
Host: 192.168.0.10
Origin: http://192.168.0.10
Referer: http://192.168.0.10/
User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36
X-Requested-With: XMLHttpRequest'''
    
class URLlist:
    read_current_setup = "http://192.168.0.10/read_file.php?filename=defaultParameters.txt&time=0"
    write_file = "http://192.168.0.10/write_file.php"
class HTTPMethods:
    
    def get_request(self,header = Header.standard_header,url=''):
        PARAMS = Parser.parse_string_to_dict(header)

        # sending get request and saving the response as response object 
        response = requests.get(url = url, params = PARAMS) 
        return response
    def post_request(self, header = Header.post_php_header,url = '',data = ''):
        PARAMS = Parser.parse_string_to_dict(header)
        r = requests.post(url = url, data = data, headers=PARAMS ) 
    
class POSTData(Data):
    def write_file(id):
         return '''a=exec+1%3E+log.txt%0Arm+patterns%2F*.csv%0Arm+patterns%2F*.bmp%0Arm+patterns%2F*.sdf%0Arm+tmp%2F*%0Arm+*.bmp%0Aecho+%22Generating+Patterns...%22+%3E+tmp%2Fscript_status{n}.txt%0Adlp_nirscan+-A51+-Z1726+-N110%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+tmp%2Fscript_status{n}.txt%0A+exit+0%0A+fi%0Acd+patterns%2F%0Aecho+%22Preparing+Scan+Solution...%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0Adlp_nirscan+-Pscan.sdf%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0A+exit+0%0A+fi%0Aecho+%22Loading+Patterns...%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0Acount%3D0%0Afor+entry+in+*%0Ado%0Acase+%24entry+in+*.bmp)%0Aconvert+%24entry+-virtual-pixel+Black++-filter+point+-interpolate+nearestneighbor+-distort+polynomial+%222+%24(cat+%2Fusr%2Fshare%2Fmatrix-gui-2.0%2Fcalibration_data%2Fcontrol_points.txt)%22+scan_img.bmp%0Amv+scan_img.bmp+%24entry%0Acount%3D%24((count%2B1))%3B%3B%0Aesac%0Adone%0Adlp_nirscan+-l%24count%0A%0Acd+..+%0Adlp_nirscan+-S110+-E1400+-fcustom_scan%0A%23Using+a+more+unique+string+to+detect+if+the+script+is+completed%0Aecho+%22_%3F!!SCRIPT_COMPLETED!!%3F_%22+%3E+tmp%2Fscript_status{n}.txt&filename=scan.sh'''.format(n=str(id))
    

In [104]:
## Get current settings
class APIClient(HTTPMethods):
    URLlist = URLlist()
    def get_current_settings(self,header_list = ['lower_lim','upper_lim','resolution(nm)','measurement_time(ms)','number_of_scans','number_of_points','unknown_1','unknown_2']):
        response = self.get_request(header = Header.standard_header,url = URLlist.read_current_setup)
        value_list = response.text.split()[0].strip('\"').split('\\n')
        return dict(zip(header_list,value_list))
    def post_write_file(self):
        n = np.random.randint(100,1000)
        data = POSTData.write_file(333)
        response = self.post_request(header = Header.post_php_header ,url = URLlist.write_file ,data = '')

In [81]:
API = APIClient()
API.get_current_settings()

{'lower_lim': '1350',
 'upper_lim': '2450',
 'resolution(nm)': '10',
 'measurement_time(ms)': '1771',
 'number_of_scans': '11',
 'number_of_points': '110',
 'unknown_1': '51',
 'unknown_2': '1726'}

In [82]:
APIClient.URLlist.read_current_setup

'http://192.168.0.10/read_file.php?filename=defaultParameters.txt&time=0'

1072

In [126]:
params = {'a': '''exec+1%3E+log.txt%0Arm+patterns%2F*.csv%0Arm+patterns%2F*.bmp%0Arm+patterns%2F*.sdf%0Arm+tmp%2F*%0Arm+*.bmp%0Aecho+%22Generating+Patterns...%22+%3E+tmp%2Fscript_status{n}.txt%0Adlp_nirscan+-A51+-Z1726+-N110%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+tmp%2Fscript_status{n}.txt%0A+exit+0%0A+fi%0Acd+patterns%2F%0Aecho+%22Preparing+Scan+Solution...%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0Adlp_nirscan+-Pscan.sdf%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0A+exit+0%0A+fi%0Aecho+%22Loading+Patterns...%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0Acount%3D0%0Afor+entry+in+*%0Ado%0Acase+%24entry+in+*.bmp)%0Aconvert+%24entry+-virtual-pixel+Black++-filter+point+-interpolate+nearestneighbor+-distort+polynomial+%222+%24(cat+%2Fusr%2Fshare%2Fmatrix-gui-2.0%2Fcalibration_data%2Fcontrol_points.txt)%22+scan_img.bmp%0Amv+scan_img.bmp+%24entry%0Acount%3D%24((count%2B1))%3B%3B%0Aesac%0Adone%0Adlp_nirscan+-l%24count%0A%0Acd+..+%0Adlp_nirscan+-S110+-E1400+-fcustom_scan%0A%23Using+a+more+unique+string+to+detect+if+the+script+is+completed%0Aecho+%22_%3F!!SCRIPT_COMPLETED!!%3F_%22+%3E+tmp%2Fscript_status{n}.txt'''.format(n=434),
         'filename':'scan.sh'}
encoded_params = urllib.parse.urlencode(params, doseq=True)

In [133]:
encoded_params = '''a=exec+1%3E+log.txt%0Arm+patterns%2F*.csv%0Arm+patterns%2F*.bmp%0Arm+patterns%2F*.sdf%0Arm+tmp%2F*%0Arm+*.bmp%0Aecho+%22Generating+Patterns...%22+%3E+tmp%2Fscript_status{n}.txt%0Adlp_nirscan+-A51+-Z1726+-N110%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+tmp%2Fscript_status{n}.txt%0A+exit+0%0A+fi%0Acd+patterns%2F%0Aecho+%22Preparing+Scan+Solution...%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0Adlp_nirscan+-Pscan.sdf%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0A+exit+0%0A+fi%0Aecho+%22Loading+Patterns...%22+%3E+..%2Ftmp%2Fscript_status{n}.txt%0Acount%3D0%0Afor+entry+in+*%0Ado%0Acase+%24entry+in+*.bmp)%0Aconvert+%24entry+-virtual-pixel+Black++-filter+point+-interpolate+nearestneighbor+-distort+polynomial+%222+%24(cat+%2Fusr%2Fshare%2Fmatrix-gui-2.0%2Fcalibration_data%2Fcontrol_points.txt)%22+scan_img.bmp%0Amv+scan_img.bmp+%24entry%0Acount%3D%24((count%2B1))%3B%3B%0Aesac%0Adone%0Adlp_nirscan+-l%24count%0A%0Acd+..+%0Adlp_nirscan+-S110+-E1400+-fcustom_scan%0A%23Using+a+more+unique+string+to+detect+if+the+script+is+completed%0Aecho+%22_%3F!!SCRIPT_COMPLETED!!%3F_%22+%3E+tmp%2Fscript_status{n}.txt&filename=scan.sh'''.format(n=str(np.random.randint(100,1000)))

In [134]:
encoded_params == target

False

In [129]:
# import difflib
set(encoded_params.split('+')).difference(set(target.split('+')))

{'a=exec%2B1%253E%2Blog.txt%250Arm%2Bpatterns%252F%2A.csv%250Arm%2Bpatterns%252F%2A.bmp%250Arm%2Bpatterns%252F%2A.sdf%250Arm%2Btmp%252F%2A%250Arm%2B%2A.bmp%250Aecho%2B%2522Generating%2BPatterns...%2522%2B%253E%2Btmp%252Fscript_status434.txt%250Adlp_nirscan%2B-A51%2B-Z1726%2B-N110%250Aif%2B%255B%2B%2524%253F%2B-eq%2B255%2B%255D%253B%250A%2Bthen%250A%2Becho%2B%2522failed%2522%2B%253E%2Btmp%252Fscript_status434.txt%250A%2Bexit%2B0%250A%2Bfi%250Acd%2Bpatterns%252F%250Aecho%2B%2522Preparing%2BScan%2BSolution...%2522%2B%253E%2B..%252Ftmp%252Fscript_status434.txt%250Adlp_nirscan%2B-Pscan.sdf%250Aif%2B%255B%2B%2524%253F%2B-eq%2B255%2B%255D%253B%250A%2Bthen%250A%2Becho%2B%2522failed%2522%2B%253E%2B..%252Ftmp%252Fscript_status434.txt%250A%2Bexit%2B0%250A%2Bfi%250Aecho%2B%2522Loading%2BPatterns...%2522%2B%253E%2B..%252Ftmp%252Fscript_status434.txt%250Acount%253D0%250Afor%2Bentry%2Bin%2B%2A%250Ado%250Acase%2B%2524entry%2Bin%2B%2A.bmp%29%250Aconvert%2B%2524entry%2B-virtual-pixel%2BBlack%2B%2B-filte

In [130]:
target

'a=exec+1%3E+log.txt%0Arm+patterns%2F*.csv%0Arm+patterns%2F*.bmp%0Arm+patterns%2F*.sdf%0Arm+tmp%2F*%0Arm+*.bmp%0Aecho+%22Generating+Patterns...%22+%3E+tmp%2Fscript_status434.txt%0Adlp_nirscan+-A51+-Z1726+-N110%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+tmp%2Fscript_status434.txt%0A+exit+0%0A+fi%0Acd+patterns%2F%0Aecho+%22Preparing+Scan+Solution...%22+%3E+..%2Ftmp%2Fscript_status434.txt%0Adlp_nirscan+-Pscan.sdf%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+..%2Ftmp%2Fscript_status434.txt%0A+exit+0%0A+fi%0Aecho+%22Loading+Patterns...%22+%3E+..%2Ftmp%2Fscript_status434.txt%0Acount%3D0%0Afor+entry+in+*%0Ado%0Acase+%24entry+in+*.bmp)%0Aconvert+%24entry+-virtual-pixel+Black++-filter+point+-interpolate+nearestneighbor+-distort+polynomial+%222+%24(cat+%2Fusr%2Fshare%2Fmatrix-gui-2.0%2Fcalibration_data%2Fcontrol_points.txt)%22+scan_img.bmp%0Amv+scan_img.bmp+%24entry%0Acount%3D%24((count%2B1))%3B%3B%0Aesac%0Adone%0Adlp_nirscan+-l%24count%0A%0Acd+..+%0Adlp_n

In [125]:
encoded_params

'a=exec+1%3E+log.txt%0Arm+patterns%2F%2A.csv%0Arm+patterns%2F%2A.bmp%0Arm+patterns%2F%2A.sdf%0Arm+tmp%2F%2A%0Arm+%2A.bmp%0Aecho+%22Generating+Patterns...%22+%3E+tmp%2Fscript_status434.txt%0Adlp_nirscan+-A51+-Z1726+-N110%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+tmp%2Fscript_status434.txt%0A+exit+0%0A+fi%0Acd+patterns%2F%0Aecho+%22Preparing+Scan+Solution...%22+%3E+..%2Ftmp%2Fscript_status434.txt%0Adlp_nirscan+-Pscan.sdf%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+..%2Ftmp%2Fscript_status434.txt%0A+exit+0%0A+fi%0Aecho+%22Loading+Patterns...%22+%3E+..%2Ftmp%2Fscript_status434.txt%0Acount%3D0%0Afor+entry+in+%2A%0Ado%0Acase+%24entry+in+%2A.bmp%29%0Aconvert+%24entry+-virtual-pixel+Black++-filter+point+-interpolate+nearestneighbor+-distort+polynomial+%222+%24%28cat+%2Fusr%2Fshare%2Fmatrix-gui-2.0%2Fcalibration_data%2Fcontrol_points.txt%29%22+scan_img.bmp%0Amv+scan_img.bmp+%24entry%0Acount%3D%24%28%28count%2B1%29%29%3B%3B%0Aesac%0Adone%0Adlp_nirscan+-l

'a=exec+1%3E+log.txt%0Arm+patterns%2F*.csv%0Arm+patterns%2F*.bmp%0Arm+patterns%2F*.sdf%0Arm+tmp%2F*%0Arm+*.bmp%0Aecho+%22Generating+Patterns...%22+%3E+tmp%2Fscript_status333.txt%0Adlp_nirscan+-A51+-Z1726+-N110%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+tmp%2Fscript_status333.txt%0A+exit+0%0A+fi%0Acd+patterns%2F%0Aecho+%22Preparing+Scan+Solution...%22+%3E+..%2Ftmp%2Fscript_status333.txt%0Adlp_nirscan+-Pscan.sdf%0Aif+%5B+%24%3F+-eq+255+%5D%3B%0A+then%0A+echo+%22failed%22+%3E+..%2Ftmp%2Fscript_status333.txt%0A+exit+0%0A+fi%0Aecho+%22Loading+Patterns...%22+%3E+..%2Ftmp%2Fscript_status333.txt%0Acount%3D0%0Afor+entry+in+*%0Ado%0Acase+%24entry+in+*.bmp)%0Aconvert+%24entry+-virtual-pixel+Black++-filter+point+-interpolate+nearestneighbor+-distort+polynomial+%222+%24(cat+%2Fusr%2Fshare%2Fmatrix-gui-2.0%2Fcalibration_data%2Fcontrol_points.txt)%22+scan_img.bmp%0Amv+scan_img.bmp+%24entry%0Acount%3D%24((count%2B1))%3B%3B%0Aesac%0Adone%0Adlp_nirscan+-l%24count%0A%0Acd+..+%0Adlp_n

In [ ]:
dlp_nirscan -A51 -Z1726 -N100
